In [2]:
#Downloading the dogs and cats dataset from Kaggle
!kaggle competitions download -c dogs-vs-cats -p ./data/

100%|███████████████████████████████████████▉| 811M/812M [00:53<00:00, 21.7MB/s]
100%|████████████████████████████████████████| 812M/812M [00:53<00:00, 15.9MB/s]


In [6]:
# Unzip the dataset
import zipfile

file = "data/dogs-vs-cats.zip"
data_location = "data/"

zip_files = ['data/test1.zip','data/train.zip']

In [5]:
# Unzipping
zipped = zipfile.ZipFile(file, 'r')
zipped.extractall(data_location)
zipped.close()

In [7]:
# Unzipping again
for zip in zip_files:
    zipped = zipfile.ZipFile(zip, 'r')
    zipped.extractall(data_location)
    zipped.close()

#Renaming the test1 folder to test
import os 
os.rename('data/test1', 'data/test')

In [ ]:
import shutil
train_dir = 'data/train/'

for filename in os.listdir(train_dir):
    # checking if it is a file
    if 'cat' in filename:
        # shutil.move(filename, 'data/train/cats/' + filename)
        os.system('mv ' + train_dir + filename + ' ' + train_dir + 'cats/' + filename)
        print("y")
    elif 'dog' in filename:
        # shutil.move(filename, 'data/train/dogs/' + filename)
        os.system('mv ' + train_dir + filename + ' ' + train_dir + 'dogs/' + filename)
        print("n")
    else:
        print('No cat or dog in the file')

In [4]:
# Importing the inception V3 model
from tensorflow.keras.applications.inception_v3 import InceptionV3
# Downloading the weights
import urllib

weights = 'https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'

weights_file = 'inception_v3.h5'
urllib.request.urlretrieve(weights, weights_file)
# Loading the weights
pretrained_model = InceptionV3(
    input_shape=(150, 150, 3),
    include_top=False,
    weights=None
)

pretrained_model.load_weights(weights_file)
pretrained_model.summary()
for layer in pretrained_model.layers:
    layer.trainable = False

last_layer = pretrained_model.get_layer('mixed7')
print('last layer output shape: ', last_layer.output_shape)
last_output = last_layer.output
from tensorflow.keras.layers import Flatten, Dense

x = Flatten()(last_output)
x = Dense(1024, activation='relu')(x)
x = Dense(1, activation='sigmoid')(x)
from tensorflow.keras import Model

model = Model(pretrained_model.input, x)
from tensorflow.keras.optimizers import RMSprop

model.compile(optimizer=RMSprop(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])
model.summary()
# Image augmentation
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_dir = 'data/train/'

#Scale by 1/255
train_datagen = ImageDataGenerator(
    rescale=1/255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
    )

# Flow from directory
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    class_mode='binary'
)
valid = 'data/test/'

# Validation Image generator
from tensorflow.keras.preprocessing.image import ImageDataGenerator

#Scale by 1/255
valid_datagen = ImageDataGenerator(rescale=1/255)

# Flow from directory
valid_generator = train_datagen.flow_from_directory(
    valid,
    target_size=(150, 150),
    class_mode='binary'
)
# Training the model
history = model.fit(
    train_generator,
    epochs=15,
    validation_data=valid_generator,
)

Metal device set to: Apple M1 Pro

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



2022-06-03 18:50:52.155396: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-06-03 18:50:52.156632: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Model: "inception_v3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 150, 150, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 74, 74, 32)   864         ['input_1[0][0]']                
                                                                                                  
 batch_normalization (BatchNorm  (None, 74, 74, 32)  96          ['conv2d[0][0]']                 
 alization)                                                                                       
                                                                                       

2022-06-03 18:50:54.573738: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


Epoch 1/15


2022-06-03 18:50:55.474998: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


782/782 [==============================] - 94s 119ms/step - loss: 0.2035 - accuracy: 0.9209
Epoch 2/15
289/782 [==========>...................] - ETA: 59s - loss: 0.1754 - accuracy: 0.9330

KeyboardInterrupt: 